In [ ]:
# 官方文档教程
# https://python.langchain.com/docs/tutorials/chatbot/

In [ ]:
# 构建聊天机器人


In [ ]:
# 概述
# 我们将通过一个示例，展示如何设计和实现一个基于 LLM 的聊天机器人。该聊天机器人将能够与聊天模型进行对话，并记住之前的交互。

In [ ]:
# 设置
# Jupyter Notebook
# 本指南（以及文档中的大多数其他指南）均使用 Jupyter Notebook ，并假设读者也使用 Jupyter Notebook。Jupyter Notebook 非常适合学习如何使用 LLM 系统，因为经常会出现问题（例如意外输出、API 故障等），而在交互式环境中阅读指南是更好地理解这些系统的绝佳方式。

In [ ]:
# 安装
# 本教程需要 langchain-core 和 langgraph 。本指南需要 langgraph >= 0.2.28 。
# conda install langchain-core langgraph>0.2.27 -c conda-forge

In [ ]:
# LangSmith
# 使用 LangChain 构建的许多应用程序都包含多个步骤，需要多次调用 LLM 函数。随着这些应用程序变得越来越复杂，能够检查链或代理内部究竟发生了什么变得至关重要。最好的方法是使用 LangSmith 。

In [ ]:
# 通过上面的链接注册后，请确保设置环境变量以开始记录跟踪：

# export LANGSMITH_TRACING="true"
# export LANGSMITH_API_KEY="..."

In [ ]:
# 或者，如果在笔记本中，您可以使用以下方式设置它们：

# import getpass
# import os

# os.environ["LANGSMITH_TRACING"] = "true"
# os.environ["LANGSMITH_API_KEY"] = getpass.getpass()

In [ ]:
# 快速入门
# 首先，让我们学习如何单独使用语言模型。LangChain 支持多种不同的语言模型，您可以互换使用 - 请在下方选择您想要使用的模型！
# 选择聊天模型 ：
# Google Gemini ▾
# pip install -qU "langchain[google-genai]"

In [1]:
import getpass
import os

if not os.environ.get("GOOGLE_API_KEY"):
  os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")

from langchain.chat_models import init_chat_model

model = init_chat_model("gemini-2.0-flash", model_provider="google_genai")

In [2]:
# 首先让我们直接使用模型。ChatModel 是 ChatModel “Runnables”的实例，这意味着它们暴露了一个用于交互的标准接口。为了简单地调用模型，我们可以将消息列表传递给 .invoke 方法。

In [3]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="Hi! I'm Miku")])

AIMessage(content="Hi Miku! It's nice to meet you. How can I help you today?", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--04d948fe-7cb2-4500-b794-a409348980b3-0', usage_metadata={'input_tokens': 7, 'output_tokens': 20, 'total_tokens': 27, 'input_token_details': {'cache_read': 0}})

In [4]:
# 该模型本身没有任何状态概念。例如，如果你问一个后续问题：

In [5]:
model.invoke([HumanMessage(content="What's my name?")])

AIMessage(content="As a large language model, I have no way of knowing your name. You haven't told me!", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--d4e62309-c9bb-4c04-aa3f-68fcfa5a4091-0', usage_metadata={'input_tokens': 6, 'output_tokens': 23, 'total_tokens': 29, 'input_token_details': {'cache_read': 0}})

In [ ]:
# 让我们看一下 LangSmith 跟踪示例
# 我们发现它没有将之前的对话内容纳入上下文，也无法回答问题。这给聊天机器人带来了糟糕的体验！
# 为了解决这个问题，我们需要将整个对话历史记录传入模型。让我们看看这样做会发生什么：


In [6]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="Hi! I'm Miku"),
        AIMessage(content="Hello Miku! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ]
)

AIMessage(content='Your name is Miku. You just told me! 😊', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--a079d517-e392-43a2-9bff-731199c61166-0', usage_metadata={'input_tokens': 24, 'output_tokens': 13, 'total_tokens': 37, 'input_token_details': {'cache_read': 0}})

In [ ]:
# 现在我们可以看到我们得到了良好的响应！

In [ ]:
# 这是聊天机器人对话交互能力的基本理念。那么，我们如何才能最好地实现这一点呢？

In [7]:
# 消息持久化
# LangGraph 实现了内置持久层，使其成为支持多轮对话的聊天应用程序的理想选择。
# 将我们的聊天模型包装在一个最小的 LangGraph 应用程序中，使我们能够自动保存消息历史记录，从而简化多轮应用程序的开发。
# LangGraph 自带一个简单的内存检查点，我们将在下面使用它。更多详细信息，包括如何使用不同的持久化后端（例如 SQLite 或 Postgres），请参阅其文档 。

In [8]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

# Define a new graph
workflow = StateGraph(state_schema=MessagesState)


# Define the function that calls the model
def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    return {"messages": response}


# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [9]:
# 现在我们需要创建一个 config ，每次都传递给可运行对象。此配置包含一些不直接包含在输入中但仍然有用的信息。在本例中，我们希望包含一个 thread_id 。它应该如下所示：

In [10]:
config = {"configurable": {"thread_id": "abc123"}}

In [11]:
# 这使我们能够使用单个应用程序支持多个对话线程，当您的应用程序有多个用户时，这是一个常见的要求。
# 然后我们可以调用该应用程序：



In [12]:
query = "Hi! I'm Miku."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()  # output contains all messages in state

================================== Ai Message ==================================

Hi Miku! It's nice to meet you. How can I help you today?


In [13]:
query = "What's my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Your name is Miku. You just told me! 😊


In [ ]:
# 太棒了！我们的聊天机器人现在记住了我们的一些事情。如果我们修改配置，引用不同的 thread_id ，就能看到它重新开始对话了。

In [14]:
config = {"configurable": {"thread_id": "abc234"}}

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

As a large language model, I don't know your name. I have no memory of past conversations and don't have access to personal information. You haven't told me your name.


In [15]:
# 但是，我们总是可以回到原始对话（因为我们将其保存在数据库中）

In [16]:
config = {"configurable": {"thread_id": "abc123"}}

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

You told me your name is Miku. Is that still correct?


In [ ]:
# 这就是我们如何支持聊天机器人与许多用户进行对话！

In [ ]:
# tip  提示
# 对于异步支持，将 call_model 节点更新为异步函数，并在调用应用程序时使用 .ainvoke ：


In [17]:
# Async function for node:
async def call_model(state: MessagesState):
    response = await model.ainvoke(state["messages"])
    return {"messages": response}


# Define graph as before:
workflow = StateGraph(state_schema=MessagesState)
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)
app = workflow.compile(checkpointer=MemorySaver())

# Async invocation:
output = await app.ainvoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

As a large language model, I have no memory of past conversations. Therefore, I don't know your name. You haven't told me!


In [ ]:
# 目前，我们所做的只是在模型周围添加了一个简单的持久层。我们可以通过添加提示模板来使聊天机器人变得更加复杂和个性化。

In [ ]:
# Prompt templates(提示模板)
# 提示模板有助于将原始用户信息转换为 LLM 可以处理的格式。
# 在本例中，原始用户输入只是一条消息，我们会将其传递给 LLM。
# 现在让我们让它更复杂一些。
# 首先，让我们添加一条包含一些自定义指令的系统消息（但仍然接受消息作为输入）。
# 接下来，除了消息之外，我们还将添加更多输入。


In [ ]:
# 为了添加系统消息，我们将创建一个 ChatPromptTemplate 。我们将利用 MessagesPlaceholder 来传递所有消息。



In [18]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You talk like a pirate. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [19]:
# 我们现在可以更新我们的应用程序以包含此模板：

In [20]:
workflow = StateGraph(state_schema=MessagesState)


def call_model(state: MessagesState):
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    return {"messages": response}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [ ]:
# 我们以同样的方式调用应用程序：

In [21]:
config = {"configurable": {"thread_id": "abc345"}}
query = "Hi! I'm LuoTianYi."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Ahoy there, LuoTianYi! A pleasure to meet ye on the digital seas! What brings ye to these waters? I be ready to lend a hand, or at least spin a yarn or two.


In [22]:
query = "What is my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Shiver me timbers, ye be testin' me memory, aye? Yer name be LuoTianYi, as ye told me just a moment ago! Don't be makin' this ol' salt doubt his senses!


In [ ]:
# 太棒了！现在让我们把提示符变得更复杂一些。假设提示符模板现在看起来像这样：


In [23]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [24]:
# 请注意，我们在提示框中添加了新的 language 输入。我们的应用现在有两个参数——输入 messages 和 language 。我们应该更新应用的状态以反映这一点：


In [25]:
from typing import Sequence

from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict


class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    language: str


workflow = StateGraph(state_schema=State)


def call_model(state: State):
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    return {"messages": [response]}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [26]:
config = {"configurable": {"thread_id": "abc456"}}
query = "Hi! I'm LuoTianYi."
language = "Chinese"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

你好，洛天依！很高兴认识你。有什么我可以帮你的吗？


In [ ]:
# 请注意，整个状态是持久的，因此如果不需要更改，我们可以省略 language 等参数：


In [27]:
query = "What is my name?"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

你的名字是洛天依。


In [ ]:
# Managing Conversation History（管理对话历史记录）
# 构建聊天机器人时，需要理解的一个重要概念是如何管理对话历史记录。如果不加以管理，消息列表将无限增长，并可能溢出 LLM 的上下文窗口。因此，添加一个限制传入消息大小的步骤非常重要。
# 重要的是，您需要在提示模板之前但在从消息历史记录中加载以前的消息之后执行此操作。
# 我们可以在提示前添加一个简单的步骤来适当修改 messages 键，然后将该新链包装在消息历史记录类中。
# LangChain 自带一些内置助手，用于管理消息列表 。在本例中，我们将使用 trim_messages 助手来减少发送给模型的消息数量。修剪器允许我们指定要保留的标记数量，以及其他参数，例如是否要始终保留系统消息以及是否允许部分消息：


In [28]:
from langchain_core.messages import SystemMessage, trim_messages

trimmer = trim_messages(
    max_tokens=65,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content="hi! I'm bob", additional_kwargs={}, response_metadata={}),
 AIMessage(content='hi!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [ ]:
# 为了在我们的链中使用它，我们只需要在将 messages 输入传递给提示之前运行修剪器。


In [31]:
workflow = StateGraph(state_schema=State)


def call_model(state: State):
    trimmed_messages = trimmer.invoke(state["messages"])
    prompt = prompt_template.invoke(
        {"messages": trimmed_messages, "language": state["language"]}
    )
    response = model.invoke(prompt)
    return {"messages": [response]}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [32]:
# 现在，如果我们尝试询问模型我们的名字，它将不会知道，因为我们修剪了聊天记录的那一部分：


In [33]:
config = {"configurable": {"thread_id": "abc567"}}
query = "What is my name?"
language = "Chinese"

input_messages = messages + [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

作为一个AI，我无法知道你的名字。你没有告诉我你的名字。


In [ ]:
# 但如果我们询问最近几条消息中的信息，它会记住：

In [34]:
config = {"configurable": {"thread_id": "abc678"}}
query = "What math problem did I ask?"
language = "Chinese"

input_messages = messages + [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

你问的数学问题是：2 + 2 等于多少？


In [ ]:
# 如果您看一下 LangSmith，您就可以清楚地看到 LangSmith 跟踪中幕后发生的事情。


In [ ]:
# Streaming(流式传输)
# 现在，我们已经有了一个可以运行的聊天机器人。然而，对于聊天机器人应用程序来说，一个非常重要的用户体验考虑因素是流式传输。LLM 有时可能需要一段时间才能响应，因此为了提升用户体验，大多数应用程序都会在每个令牌生成时将其流式传输回来。这让用户能够看到进度。
# 默认情况下，我们的 LangGraph 应用程序中的 .stream 会流式传输应用程序步骤——在本例中，是模型响应的单个步骤。设置 stream_mode="messages" 允许我们改为流式传输输出令牌：



In [37]:
config = {"configurable": {"thread_id": "abc789"}}
query = "Hi I'm Ultraman, please tell me a joke."
language = "Chinese"

input_messages = [HumanMessage(query)]
for chunk, metadata in app.stream(
    {"messages": input_messages, "language": language},
    config,
    stream_mode="messages",
):
    if isinstance(chunk, AIMessage):  # Filter to just model responses
        print(chunk.content, end="|")

你好|奥|特曼！

这里有个笑话给你：

为什么奥特曼总是|赢？

因为他总是“奥特”一把！ (因为他总是“all| out” 一把， “奥特” 和 “all out” 发音相似)

希望你喜欢！
|